In [17]:
import gym
import numpy as np
import tensorflow as tf

In [18]:
env = gym.make("CartPole-v0")

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [19]:
num_input_neurons = len(env.reset())
num_ouptut_neurons = env.action_space.n
hidden_layer_dimensions = [16, 32, 32]

learning_rate = 0.0001

In [20]:
class DQN:
    def __init__(self,
                 session,
                 input_size,
                 hidden_layer_sizes,
                 output_size,
                 learning_rate):
        
        self.session = session
        self.input_size = input_size
        self.hidden_layer_sizes = hidden_layer_sizes
        self.output_size = output_size
        self.learning_rate = learning_rate
        
        self.input_layer = tf.placeholder(shape=[1, self.input_size],
                                     dtype=tf.float32)
        
        net = self.input_layer
        for layer_size in self.hidden_layer_sizes:
            net = tf.layers.dense(net, layer_size, activation=tf.nn.relu)
        
        self.output = tf.layers.dense(net, self.output_size, activation=None)
        
        # Placeholder for expected q-values
        self.y = tf.placeholder(shape=[1, self.output_size], dtype=tf.float32)
        
        # Using the loss method provided by tf directly
        self.loss = tf.losses.mean_squared_error(self.y, self.output)
        
        self.optimizer = tf.train.AdamOptimizer(
            learning_rate=self.learning_rate).minimize(self.loss)
        
        def predict(self, state):
            return self.session.run(self.output,
                                    feed_dict={self.input: np.reshape(state, [-1, self.input_size])})

        def update(self, state, y):
            return self.session.run([self.loss, self.optimizer],
                                     feed_dict={
                                             self.input: state,
                                             self.y: y
                                     })

In [21]:
with tf.Session() as sess:
    main_dqn = DQN(session=sess,
                   input_size=num_input_neurons,
                   hidden_layer_sizes=hidden_layer_dimensions,
                   output_size=num_ouptut_neurons,
                   learning_rate=learning_rate)
    
    target_dqn = DQN(session=sess,
                     input_size=num_input_neurons,
                     hidden_layer_sizes=hidden_layer_dimensions,
                     output_size=num_ouptut_neurons,
                     learning_rate=learning_rate)
    
    sess.run(tf.global_variables_initializer())
    
    # Make the weights/config identical to begin with
    # How?